In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

#Library
# ---
# For scrape
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Options driver
from selenium.webdriver.chrome.options import Options

# For dataframes
import pandas as pd
from pandas import json_normalize
import os

# For simulate human behavior.
import time
from time import sleep
import random
from selenium.webdriver.common.by import By

# Clear data
import unidecode
#unaccented_string = unidecode.unidecode(accented_string)

# Manipulate Json files
import json
import re
import numpy as np

In [ ]:
folder = "group_3"
relative_path = "..\..\_output\sbs\municipal/" + folder

if not os.path.exists(relative_path):
    os.makedirs(relative_path)

In [85]:
# Setting
options= Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# we open our page
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

C:\Users\DELL\AppData\Local\Temp\ipykernel_3924\2882332421.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)


In [86]:
# Dropdown button list
dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio")
dropdown_button_year.click()

In [87]:
# List of years options
list_year = driver.find_element( By.ID,  "ctl00_cphContent_rAnio_DropDown" )
years_list = list_year.find_elements( By.TAG_NAME, 'li')
list_year_options = len(years_list)

In [88]:
# Dropdown button list
dropdown_button_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes")
dropdown_button_month.click()

In [89]:
# # List of months options
list_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes_DropDown" )
months_list = list_month.find_elements( By.TAG_NAME, 'li')
list_month_options = len(months_list)
driver.find_element( By.ID, "ctl00_cphContent_btnConsultaMensual").click()
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

In [90]:
# We have 2 currencies: national and  foreign. we iterate in this too. 
driver.find_element( By.ID, "ctl00_cphContent_lbtnMn").click()
driver.find_element( By.ID, "ctl00_cphContent_lbtnMex").click()
currencies = ["Na", "Fo"]

In [91]:
year_numbers_opt = np.array( [1, 0 ] + np.arange(1, list_year_options).tolist() )
month_numbers_opt = np.array( [1, 0 ] + np.arange(1, list_month_options).tolist() )

In [92]:
#Check for th values of a number of arrays
month_numbers_opt[2:13]


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [93]:
years_info_list = []
months_info_list = []
tables_info_list = []

#years start since 2023 to the start of the list
for year_number in year_numbers_opt[1:22]:
    
    # Dropdown button list
    dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio") 
    dropdown_button_year.click()

    # List of years options
    time.sleep(0.5)
    driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{year_number + 1 }]" ).click()
    year_text = driver.find_element( By.ID, "ctl00_cphContent_rAnio").text

    
#months start since january to december        
    for month_number in month_numbers_opt[1:13]:
        
        # Dropdown button list
        dropdown_button_month = driver.find_element( By.ID, "ctl00_cphContent_rMes") 
        dropdown_button_month.click()

        # # List of months options
        time.sleep(0.5)
        driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{month_number + 1 }]" ).click()
        month_text = driver.find_element( By.ID, "ctl00_cphContent_rMes").text
        print( year_text )
        print( month_text )
        
        # Click consultar
        driver.find_element( By.ID,"ctl00_cphContent_btnConsultaMensual").click()
        time.sleep(5)
        
        year_float = int(year_text)
        
        # It itiretes for tyoe of currency.
        for currency in currencies:
            
            try:
                driver.find_element( By.ID, f"ctl00_cphContent_lbtn{currency}" ).click()
                time.sleep(5)
                print(currency)
            except:
                print("No currency tipo 1")
            try:

                driver.find_element( By.ID, f"ctl00_cphContent_lbtn{currency}ASP" ).click()
                time.sleep(5)
                print(currency)
            except:
                print("No currency tipo 2")

            # Identifying the tables in the web page, there are three types of formats in all the period of interest
        
            try:
                driver.find_element( By.ID, f"ctl00_cphContent_lbtn{currency}" ).click()
                time.sleep(5)
                print(currency)
                
            except:
                print("No currency tipo 1")

            try:

                driver.find_element( By.ID, f"ctl00_cphContent_lbtn{currency}ASP" ).click()
                time.sleep(5)
                print(currency)
            except:
                print("No currency tipo 2")

            # Identifying the tables in the web page, there are three types of formats in all the period of interest
            # Type 1 table - Changing the frame and extracting a simple table

            try:
                # Switch elements
                driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
                section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
                table_html = section_tab_table.get_attribute('outerHTML')
                table_pd = pd.read_html( table_html )

                # Cleaning tablr
                table_clean_1 = table_pd[1].iloc[5:, :].copy().reset_index( drop = True )

                # Changing switch
                driver.switch_to.default_content()

                print("Table Tipo 1 - Cambiar de Frame y una tabla simple")

            except:
                print("No table Tipo 1")
                table_clean_1 = "Null"


            try:
                # Switch elements
                driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
                section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
                table_html = section_tab_table.get_attribute('outerHTML')
                table_pd = pd.read_html( table_html )

                # generate table objects
                table0, table1 = table_pd[0], table_pd[1]
                table_clean_2 = table1.copy()

                # Number of rows of second table
                tb1_rows = table1.shape[0]
                drop_rows = np.arange( 1, tb1_rows + 2 )

                # Clean first table
                table0 = table0.drop( drop_rows, axis = 0 )

                # Assign values to new table
                col_name = table0.iloc[0, 0]
                col_values = table0.iloc[1:, 0].tolist()
                table_clean_2.insert( loc = 0, column= col_name, value = col_values)

                print("Table Tipo 2 - Cambiar de Frame y una tabla Compleja")

                 # Changing switch
                driver.switch_to.default_content()

            except:
                print("No table Tipo 2")
                table_clean_2 = "Null"


            try:
                # Get tables
                section_id_table = driver.find_element( By.ID, "ctl00_cphContent_rpgActualMn_OT" )
                table_html = section_id_table.get_attribute('outerHTML')
                table_pd = pd.read_html( table_html )

                # generate table objects
                table0, table1 = table_pd[0], table_pd[1]
                table_clean_3 = table1.copy()

                # Number of rows of second table
                tb1_rows = table1.shape[0]
                drop_rows = np.arange( 1, tb1_rows + 2 )

                # Clean first table
                table0 = table0.drop( drop_rows, axis = 0 )

                # Assign values to new table
                col_name = table0.iloc[0, 0]
                col_values = table0.iloc[1:, 0].tolist()
                table_clean_3.insert( loc = 0, column= col_name, value = col_values)

                print("Table Tipo 3 - No Cambiar de Frame y una tabla Compleja")

            except:
                print("No table Tipo 3")
                table_clean_3 = "Null"


            # Check what to save
            if ( ( isinstance( table_clean_3, str ) == False ) and ( table_clean_3.shape[1] > 5 ) ):

                # Data with no frame
                table_clean_3.to_excel( fr"..\..\..\Diplomado_PUCP\_output\sbs\municipal\group_3/table_clean_{year_text}_{month_text}_{currency}.xlsx", index = False )

                years_info_list.append( year_float)
                months_info_list.append( month_text )
                tables_info_list.append( "Table 3" )
                print("guardo3")

            elif ( ( isinstance( table_clean_2, str ) == False ) and ( table_clean_2.shape[1] > 5 ) ):

                # Data with frame and 2 tables
                table_clean_2.to_excel( fr"..\..\..\Diplomado_PUCP\_output\sbs\municipal\group_3/table_clean_{year_text}_{month_text}_{currency}.xlsx", index = False, header = False )
                # To break
                years_info_list.append( year_float)
                months_info_list.append( month_text )
                tables_info_list.append( "Table 2" )
                print("guardo2")

            elif ( ( isinstance( table_clean_1, str ) == False ) and ( table_clean_1.shape[1] > 5 ) ):
                # Data with frame and 1 table
                table_clean_1.to_excel( fr"..\..\..\Diplomado_PUCP\_output\sbs\municipal\group_3/table_clean_{year_text}_{month_text}_{currency}.xlsx", index = False , header = False )

                years_info_list.append( year_float)
                months_info_list.append( month_text )
                tables_info_list.append( "Table 1" )
                print("guardo1")

            # No table information
            # Go to a previous day
            else:
                # Day ago
                years_info_list.append( year_float)
                months_info_list.append( month_text )
                tables_info_list.append( "No Table" )

            driver.switch_to.default_content()


2023
Enero
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2023
Febrero
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
No table Tipo 3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
No table Tipo 3
2023
Marzo
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
No table Tipo 3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
No table Tipo 3
2023
Abril
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
No tab

2021
Marzo
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021
Abril
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021
Mayo
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2


2019
Febrero
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2019
Marzo
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2019
Abril
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
No currency tipo 1
No currency tipo 2
No currency tipo 1
No currency tipo 2
No table Tipo 1
No table Tipo

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=110.0.5481.104)
Stacktrace:
Backtrace:
	(No symbol) [0x00D037D3]
	(No symbol) [0x00C98B81]
	(No symbol) [0x00B9B230]
	(No symbol) [0x00BCE337]
	(No symbol) [0x00BC3253]
	(No symbol) [0x00BEB41C]
	(No symbol) [0x00BC2B96]
	(No symbol) [0x00BEB774]
	(No symbol) [0x00C01215]
	(No symbol) [0x00BEB216]
	(No symbol) [0x00BC0D97]
	(No symbol) [0x00BC253D]
	GetHandleVerifier [0x00F7ABF2+2510930]
	GetHandleVerifier [0x00FA8EC1+2700065]
	GetHandleVerifier [0x00FAC86C+2714828]
	GetHandleVerifier [0x00DB3480+645344]
	(No symbol) [0x00CA0FD2]
	(No symbol) [0x00CA6C68]
	(No symbol) [0x00CA6D4B]
	(No symbol) [0x00CB0D6B]
	BaseThreadInitThunk [0x76F97D69+25]
	RtlInitializeExceptionChain [0x77C6BB9B+107]
	RtlClearBits [0x77C6BB1F+191]
